# Toy example of small sample regression

### With OLS and Bayesian Regression

### Packages

In [15]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go
from scipy.stats import norm
import numpy as np

### SLR toy example

In [16]:
np.random.seed(0)

# Generate full dataset
n_total = 45
X_full = np.linspace(0, 10, n_total).reshape(-1, 1)
y_true = X_full.ravel() - X_full.ravel() + 7
noise = np.random.normal(0, 8, size=n_total)
y_noisy = y_true + noise

# Prepare test grid
xx = np.linspace(X_full.min() - 1, X_full.max() + 1, 200).reshape(-1, 1)
yy_true = xx.ravel() - xx.ravel() + 7

# Prepare figure
fig = go.Figure()
frames = []
sample_sizes = np.arange(5, n_total + 1, 10)  # e.g. 5, 10, 15, 20, 25, 30

for n_samples in sample_sizes:
    # Subset current data
    X = X_full[:n_samples]
    y = y_noisy[:n_samples]
    
    # Train/test split (fixed ratio)
    X_train, X_test, y_train, y_test, y_true_train, y_true_test = train_test_split(
        X, y, y_true[:n_samples], test_size=0.4, random_state=42
    )
    
    # Fit models
    ols = LinearRegression().fit(X_train, y_train)
    bayes = BayesianRidge().fit(X_train, y_train)
    
    # Predictions
    y_pred_ols = ols.predict(X_test)
    y_pred_bayes, y_std_bayes = bayes.predict(X_test, return_std=True)
    
    mse_ols = mean_squared_error(y_true_test, y_pred_ols)
    mse_bayes = mean_squared_error(y_true_test, y_pred_bayes)
    
    yy_ols = ols.predict(xx)
    yy_bayes, yy_bayes_std = bayes.predict(xx, return_std=True)
    
    # Frame for this sample size
    frames.append(go.Frame(
        name=str(n_samples),
        data=[
            # Train data
            go.Scatter(x=X_train.ravel(), y=y_train, mode='markers',
                       marker=dict(color='gray', size=8, opacity=0.6),
                       name='Train data'),
            # Test data
            go.Scatter(x=X_test.ravel(), y=y_test, mode='markers',
                       marker=dict(color='black', size=8, symbol='circle-open'),
                       name='Test data'),
            # True function
            go.Scatter(x=xx.ravel(), y=yy_true, mode='lines',
                       line=dict(color='black', dash='dash'),
                       name='True function'),
            # OLS
            go.Scatter(x=xx.ravel(), y=yy_ols, mode='lines',
                       line=dict(color='red'),
                       name=f'OLS (MSE={mse_ols:.2f})'),
            # Bayes
            go.Scatter(x=xx.ravel(), y=yy_bayes, mode='lines',
                       line=dict(color='blue'),
                       name=f'Bayes (MSE={mse_bayes:.2f})')
        ],
        layout=go.Layout(
            title_text=f"OLS vs Bayesian Regression — Sample Size: {n_samples}"
        )
    ))

# Add initial frame
fig.add_traces(frames[0].data)

# Add slider & animation settings
fig.update_layout(
    title="OLS vs Bayesian Regression",
    xaxis_title="X",
    yaxis_title="Y",
    template="plotly_white",
    sliders=[{
        "steps": [
            {"args": [[str(size)], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
             "label": str(size), "method": "animate"}
            for size in sample_sizes
        ],
        "currentvalue": {"prefix": "Sample size: "}
    }]
)

fig.frames = frames
fig.write_html("../plots/interactive_regression_1.html", include_plotlyjs='cdn')
fig.show()


### MLR toy example

In [17]:
def simulate(n_samples=80, beta_scale=5.0, random_state=0) -> dict:
    """
    Creates example data to perform OLS and Bayesian regression on

    Params 
    ------
    n_samples : int
        Defines how many samples are in the simulated data.
    beta_scale : float
        Of the non-zero coeficients, defines what their value is.
    random_state : int
        Defines random seed

    Returns : dict
        Dictionary of each predictors:
        - true coef
        - OLS estimated coef
        - Bayes estimated coef
        - MSE of OLS prediction
    """
    np.random.seed(random_state)

    # Predictors
    p = 100
    n_nonzero = 5

    # True beta
    beta = np.zeros(p)
    nonzero_idx = np.random.choice(p, n_nonzero, replace=False)
    beta[nonzero_idx] = np.random.normal(0, beta_scale, size=n_nonzero)

    # Data
    X = np.random.normal(size=(n_samples, p))
    noise = np.random.normal(0, 2.0, size=n_samples)
    y = X.dot(beta) + noise

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    # Fit models
    ols = LinearRegression().fit(X_train, y_train)
    bayes = BayesianRidge().fit(X_train, y_train)

    # Predict
    y_pred_ols = ols.predict(X_test)
    y_pred_bayes = bayes.predict(X_test)

    # Evaluate
    mse_ols = mean_squared_error(y_test, y_pred_ols)
    mse_bayes = mean_squared_error(y_test, y_pred_bayes)

    return {
        "beta_true": beta,
        "beta_ols": ols.coef_,
        "beta_bayes": bayes.coef_,
        "mse_ols": mse_ols,
        "mse_bayes": mse_bayes,
        "nonzero_idx": nonzero_idx
    }


# Slider parameter ranges
sample_sizes = [40, 80, 160, 320]
beta_scales = [1, 3, 5, 10]

# Generate all increasing combinations
results = {}
for n in sample_sizes:
    for b in beta_scales:
        results[(n, b)] = simulate(n_samples=n, beta_scale=b)

# Build Plotly figure
fig = go.Figure()

for n in sample_sizes:
    for b in beta_scales:
        res = results[(n, b)]
        # True model
        fig.add_trace(go.Scatter(
            y=res["beta_true"],
            mode='lines+markers',
            name=f"True Beta (n={n}, Beta={b})",
            visible=False,
            line=dict(color='black', dash='dot')
        ))
        # OLS model
        fig.add_trace(go.Scatter(
            y=res["beta_ols"],
            mode='lines+markers',
            name=f"OLS Beta (n={n}, Beta={b})",
            visible=False,
            line=dict(color='blue')
        ))
        # Bayes model
        fig.add_trace(go.Scatter(
            y=res["beta_bayes"],
            mode='lines+markers',
            name=f"Bayesian Beta (n={n}, Beta={b})",
            visible=False,
            line=dict(color='red')
        ))

# Create slider steps
steps = []
for i, n in enumerate(sample_sizes):
    for j, b in enumerate(beta_scales):
        step_idx = i * len(beta_scales) + j
        visible = [False] * len(fig.data)
        start = step_idx * 3
        for k in range(3):
            visible[start + k] = True

        mse_ols = results[(n, b)]["mse_ols"]
        mse_bayes = results[(n, b)]["mse_bayes"]

        step = dict(
            method="update",
            label=f"n={n}, Beta={b}",
            args=[
                {"visible": visible},
                {"title": f"True vs Estimated Beta — n={n}, Beta={b} "
                          f"<br>OLS MSE={mse_ols:.2f}, Bayes MSE={mse_bayes:.2f}"}
            ],
        )
        steps.append(step)

# Add sliders
sliders = [dict(
    active=0,
    currentvalue={"prefix": "Configuration: "},
    pad={"t": 50},
    steps=steps
)]

# Set initial visibility
for i in range(3):
    fig.data[i].visible = True

# Make plot pretty
fig.update_layout(
    sliders=sliders,
    title="True vs Estimated Coefficients (OLS vs Bayesian Ridge)",
    xaxis_title="Coefficient Index",
    yaxis_title="Coefficient Value",
    legend=dict(x=0.78, y=1, bgcolor="rgba(255,255,255,0.6)"),
    template="plotly_white"
)

# Save plot to HTML
fig.write_html("../plots/interactive_regression.html", include_plotlyjs='cdn')
fig.show()


### Variance explosion at small sample sizes

In [20]:
# Fixed parameters
beta1 = 0
sigma2 = 50
sample_sizes = [10, 20, 50, 100, 1000]

x = np.linspace(-5, 5, 400)  # plausible range for Beta Hat_1

# Compute PDFs for each sample size
pdfs = []
for n in sample_sizes:
    SSX = n  # assume Var(X)=1, so SSX =~ n
    var_hat = sigma2 / SSX
    pdf = norm.pdf(x, loc=beta1, scale=np.sqrt(var_hat))
    pdfs.append(pdf)

# Build Plotly figure
fig = go.Figure()

for i, n in enumerate(sample_sizes):
    fig.add_trace(
        go.Scatter(
            x=x,
            y=pdfs[i],
            mode="lines",
            name=f"n={n}",
            visible=False,
            line=dict(width=3, color="black")
        )
    )

# Slider setup
steps = []
for i, n in enumerate(sample_sizes):
    step = dict(
        method="update",
        label=f"{n}",
        args=[
            {"visible": [False] * len(sample_sizes)},
            {"title": f"Sampling Distribution of Beta Hat_1<br>beta_1={beta1}, sigma^2={sigma2}, n={n}"}
        ],
    )
    step["args"][0]["visible"][i] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Sample size n: "},
    pad={"t": 50},
    steps=steps
)]

# Set initial visibility and layout
fig.data[0].visible = True
fig.update_layout(
    sliders=sliders,
    title=f"Sampling Distribution of Beta Hat_1 (Beta_1={beta1}, sigma^2={sigma2})",
    xaxis_title="Beta Hat_1 (estimated slope)",
    yaxis_title="Density",
    template="plotly_white",
    width=800,
    height=500,
)

# Save to hmtl
fig.write_html("../plots/beta1_sampling_by_n.html", include_plotlyjs='cdn')
fig.show()
